In [3]:
import os
from openai import AzureOpenAI
import gradio as gr
from azure.cognitiveservices.speech import SpeechConfig, SpeechSynthesizer, AudioConfig

# Azure OpenAI 및 Cognitive Search 설정
endpoint = os.getenv("ENDPOINT_URL", "https://team10-eighti.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://team10-eighti-search.search.windows.net")
search_key = os.getenv("SEARCH_KEY", "wnalAsW6FqKRHIR6S3sUZGzNH28Lf3sBOS2ubCZsZxAzSeA205k3")
search_index = os.getenv("SEARCH_INDEX_NAME", "sign-index")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "1BajXTI5Mp0tKrs46XFGuOWjSPXKzOZSKy8e6R3qha1SVQ4lz1PFJQQJ99ALACYeBjFXJ3w3AAABACOGgPvw")

# Azure Speech Service 설정 (한국어 음성 출력)
speech_key = os.getenv("AZURE_SPEECH_KEY", "AwjVcsBAkpnrMNwkobsgJ4SSroO1GkAztrEIYp1JuMTcuKcfDR3wJQQJ99ALACYeBjFXJ3w3AAAYACOGZMDc")
speech_region = os.getenv("AZURE_SPEECH_REGION", "eastus")
speech_config = SpeechConfig(subscription=speech_key, region=speech_region)
speech_config.speech_synthesis_language = "ko-KR"  # 한국어 설정
speech_config.speech_synthesis_voice_name = "ko-KR-SunHiNeural"  # 한국어 음성
speech_synthesizer = SpeechSynthesizer(speech_config=speech_config)

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-05-01-preview",
)

# 채팅 함수 정의
def chat_with_openai(user_input, chat_history):
    try:
        # 대화 기록 포함 메시지 구성
        messages = [{"role": "system", "content": "너는 수화를 알려주는 전문가야"}]
        for user_msg, assistant_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})
        messages.append({"role": "user", "content": user_input})

        # Azure OpenAI API 호출
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False,
            extra_body={
                "data_sources": [{
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": f"{search_endpoint}",
                        "index_name": search_index,
                        "semantic_configuration": "sign-semantic",
                        "query_type": "semantic",
                        "fields_mapping": {},
                        "in_scope": True,
                        "role_information": "너는 수화를 알려주는 전문가야",
                        "filter": None,
                        "strictness": 3,
                        "top_n_documents": 5,
                        "authentication": {
                            "type": "api_key",
                            "key": f"{search_key}"
                        }
                    }
                }]
            }
        )

        # 응답 추출 및 대화 기록 업데이트
        assistant_reply = completion.choices[0].message.content.replace(' [doc1]', '').strip()  # 속성 접근 방식으로 수정
        chat_history.append((user_input, assistant_reply))  # 튜플 형태로 추가

        citations = completion.choices[0].message.context['citations'][0]['content'].split('\n')
        video_url = citations[-2]
        image_urls = citations[-3].split(' ')   # 이미지 URL이 여러 개일 수 있음

        # Azure Speech Service로 응답 읽기 (텍스트를 한국어 음성으로 변환)
        # speech_synthesizer.speak_text_async(assistant_reply)

        return chat_history, chat_history, video_url, image_urls

    except Exception as e:
        return [("Error", str(e))], chat_history

# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            gr.Markdown("# Azure OpenAI + Cognitive Search + Speech 기반 수화 챗봇")
            chatbot = gr.Chatbot()
            user_input = gr.Textbox(label="Your Message", placeholder="메시지를 입력하세요...")
            clear_button = gr.Button("Clear Chat")
        with gr.Column():
            video_display = gr.Video(label="수어 영상")
            image_display = gr.Gallery(label="수어 이미지", columns=3)

    # 대화 기록 저장
    state = gr.State([])

    # 이벤트 연결    
    user_input.submit(chat_with_openai, [user_input, state], [chatbot, state, video_display, image_display])
    clear_button.click(lambda: [], None, chatbot)
    
# 실행
demo.launch()

c:\python\Lib\site-packages\gradio\components\chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\python\Lib\site-packages\gradio\queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\python\Lib\site-packages\gradio\route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\python\Lib\site-packages\gradio\blocks.py", line 2028, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\python\Lib\site-packages\gradio\blocks.py", line 1784, in postprocess_data
    self.validate_outputs(block_fn, predictions)  # type: ignore
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\python\Lib\site-packages\gradio\blocks.py", line 1739, in validate_outputs
    raise ValueError(
ValueError: A  function (chat_with_openai) di